In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import yaml
import numpy as np
import matplotlib.pyplot as plt

from ray.rllib.algorithms.ppo import PPOConfig

from src import foundation
from src.foundation.environment_wrapper import RLLibEnvWrapper

In [13]:
config_file_path = "../experiments/free-market/config.yaml"

In [14]:
with open(config_file_path, "r") as file:
    config = yaml.safe_load(file)

In [18]:
def policy_mapping_fun(i):
    if str(i).isdigit() or i == "a":
        return "a"
    return "p"


environment_config = {
    "env_config_dict": config.get("env"),
    "num_envs_per_worker": config.get("num_envs_per_worker"),
}
trainer_config = config.get("trainer")
dummy_environment = RLLibEnvWrapper(environment_config)

agent_policy = (
    None,
    dummy_environment.observation_space,
    dummy_environment.action_space,
    config.get("agent_policy")
)

planner_policy = (
    None,
    dummy_environment.observation_space_pl,
    dummy_environment.action_space_pl,
    config.get("planner_policy"),
)

policies = {"a": agent_policy, "p": planner_policy}
if config["general"]["train_planner"]:
        policies_to_train = ["a", "p"]
else:
    policies_to_train = ["a"]


trainer_config.update(
    {
        "env_config": environment_config,
        "seed": 1,
        "multiagent": {
            "policies": policies,
            "policies_to_train": policies_to_train,
            "policy_mapping_fn": policy_mapping_fun,
        },
        "metrics_smoothing_episodes": trainer_config.get("num_workers")
        * trainer_config.get("num_envs_per_worker"),
    }
)

/home/baldoinov/Projetos/brazilian-ai-economist/.venv/lib/python3.10/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')
